In [1]:
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sqlalchemy import inspect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from connection import create_db_connection

In [2]:
# Membuat koneksi ke database
engine = create_db_connection()

# Inspect the database to get a list of tables
inspector = inspect(engine)
tables = inspector.get_table_names()

# Check if the connection was successful and if tables were retrieved
if tables:
    print("Connection to the database was successful.")
    print("Tables in the database:", tables)
else:
    print("Failed to connect to the database.")

Connection to the database was successful.
Tables in the database: ['categories', 'failed_jobs', 'migrations', 'password_reset_tokens', 'personal_access_tokens', 'product_images', 'product_reviews', 'products', 'profiles', 'subcategories', 'users']


In [3]:
# Query untuk mengambil data dari tabel products, product_reviews, dan profiles
q_product = """
    SELECT p.id, p.category_id, p.subcategory_id, p.name,pf.gender, pf.skin_type_face, pf.hair_issue, pf.skin_type_body
    FROM products p
    LEFT JOIN product_reviews pr ON p.id = pr.product_id
    LEFT JOIN profiles pf ON pr.user_id = pf.user_id
"""

# Mengambil data dari database menggunakan query
dt_product_df = pd.read_sql(q_product, engine)
display(dt_product_df.dtypes)
display(dt_product_df)

id                 int64
category_id        int64
subcategory_id     int64
name              object
gender            object
skin_type_face    object
hair_issue        object
skin_type_body    object
dtype: object

,id,category_id,subcategory_id,name,gender,skin_type_face,hair_issue,skin_type_body
0,4,2,10,Zinc Refreshing Cool - anti ketombe,Perempuan,sensitif,ketombe,kombinasi
1,4,2,10,Zinc Refreshing Cool - anti ketombe,Laki-laki,berminyak,berminyak,normal
2,5,2,10,Zinc clean active - anti ketombe,Perempuan,normal,berminyak,normal
3,6,2,10,Zinc Active fresh - anti ketombe,Laki-laki,normal,kering,normal
4,6,2,10,Zinc Active fresh - anti ketombe,Perempuan,berminyak,rontok,normal
...,...,...,...,...,...,...,...,...
574,316,3,18,Shinzui Glow Spa Honey,Laki-laki,berminyak,ketombe,kering
575,316,3,18,Shinzui Glow Spa Honey,Perempuan,sensitif,ketombe,kombinasi
576,316,3,18,Shinzui Glow Spa Honey,Laki-laki,berminyak,kering,normal
577,316,3,18,Shinzui Glow Spa Honey,Perempuan,berminyak,rontok,normal


In [4]:
# Preprocessing data teks
def preprocess_text(text):
    # Case folding
    text = str(text).lower() 
    # Punctuational removal
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenizing
    tokens = word_tokenize(text)
    # Stop words removal
    stop_words = set(stopwords.words('indonesian'))  # Menggunakan stop words bahasa Indonesia
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    # Menggabungkan kembali token menjadi kalimat
    text = ' '.join(tokens)
    return text

In [5]:
# Preprocessing kolom teks
text_columns = ['gender', 'skin_type_face', 'hair_issue', 'skin_type_body']
for column in text_columns:
    dt_product_df[column] = dt_product_df[column].apply(preprocess_text)

# Menampilkan hasil preprocessing
display(dt_product_df)

,id,category_id,subcategory_id,name,gender,skin_type_face,hair_issue,skin_type_body
0,4,2,10,Zinc Refreshing Cool - anti ketombe,perempuan,sensitif,ketombe,kombinasi
1,4,2,10,Zinc Refreshing Cool - anti ketombe,lakilaki,minyak,minyak,normal
2,5,2,10,Zinc clean active - anti ketombe,perempuan,normal,minyak,normal
3,6,2,10,Zinc Active fresh - anti ketombe,lakilaki,normal,kering,normal
4,6,2,10,Zinc Active fresh - anti ketombe,perempuan,minyak,rontok,normal
...,...,...,...,...,...,...,...,...
574,316,3,18,Shinzui Glow Spa Honey,lakilaki,minyak,ketombe,kering
575,316,3,18,Shinzui Glow Spa Honey,perempuan,sensitif,ketombe,kombinasi
576,316,3,18,Shinzui Glow Spa Honey,lakilaki,minyak,kering,normal
577,316,3,18,Shinzui Glow Spa Honey,perempuan,minyak,rontok,normal


In [6]:
# Lakukan pengelompokan berdasarkan category_id
grouped_data = dt_product_df.groupby('category_id')

# Tampilkan data untuk setiap kelompok
for category_id, group in grouped_data:
    print(f"Category ID: {category_id}")
    display(group)
    print("\n")

Category ID: 1


,id,category_id,subcategory_id,name,gender,skin_type_face,hair_issue,skin_type_body
91,46,1,1,Face Tonic Lemon,lakilaki,normal,normal,normal
92,46,1,1,Face Tonic Lemon,perempuan,minyak,minyak,kombinasi
93,46,1,1,Face Tonic Lemon,perempuan,normal,ketombe,normal
94,46,1,1,Face Tonic Lemon,lakilaki,sensitif,normal,normal
95,46,1,1,Face Tonic Lemon,perempuan,sensitif,ketombe,kombinasi
...,...,...,...,...,...,...,...,...
465,255,1,2,Herborist Raspberry & tomato,perempuan,minyak,minyak,normal
466,256,1,2,Herborist facial wash tea tree,none,none,none,none
467,257,1,2,herborist facial wash rose,none,none,none,none
468,258,1,2,herborist facial wash zaitun,perempuan,sensitif,rontok,kering




Category ID: 2


,id,category_id,subcategory_id,name,gender,skin_type_face,hair_issue,skin_type_body
0,4,2,10,Zinc Refreshing Cool - anti ketombe,perempuan,sensitif,ketombe,kombinasi
1,4,2,10,Zinc Refreshing Cool - anti ketombe,lakilaki,minyak,minyak,normal
2,5,2,10,Zinc clean active - anti ketombe,perempuan,normal,minyak,normal
3,6,2,10,Zinc Active fresh - anti ketombe,lakilaki,normal,kering,normal
4,6,2,10,Zinc Active fresh - anti ketombe,perempuan,minyak,rontok,normal
...,...,...,...,...,...,...,...,...
86,44,2,12,Putri Pure Clean Conditioner - fresh floral fr...,perempuan,minyak,minyak,normal
87,44,2,12,Putri Pure Clean Conditioner - fresh floral fr...,lakilaki,minyak,ketombe,kering
88,44,2,12,Putri Pure Clean Conditioner - fresh floral fr...,perempuan,minyak,rontok,normal
89,44,2,12,Putri Pure Clean Conditioner - fresh floral fr...,perempuan,kering,kering,kering




Category ID: 3


,id,category_id,subcategory_id,name,gender,skin_type_face,hair_issue,skin_type_body
255,114,3,16,Gatsby Durable Deo Shield,perempuan,sensitif,kering,normal
256,114,3,16,Gatsby Durable Deo Shield,perempuan,sensitif,ketombe,kombinasi
257,115,3,16,Gatsby Quick 3in1 All Body,none,none,none,none
258,116,3,16,Gatsby Shoking Ice Wave,lakilaki,minyak,ketombe,minyak
259,116,3,16,Gatsby Shoking Ice Wave,lakilaki,normal,rontok,kering
...,...,...,...,...,...,...,...,...
574,316,3,18,Shinzui Glow Spa Honey,lakilaki,minyak,ketombe,kering
575,316,3,18,Shinzui Glow Spa Honey,perempuan,sensitif,ketombe,kombinasi
576,316,3,18,Shinzui Glow Spa Honey,lakilaki,minyak,kering,normal
577,316,3,18,Shinzui Glow Spa Honey,perempuan,minyak,rontok,normal


In [7]:
# Fungsi untuk menghitung TF-IDF dan similaritas kosinus serta memberikan rekomendasi
def calculate_similarity(group):
    # Inisialisasi TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Ambil atribut untuk perhitungan (subcategory_id dan skin_type)
    attributes = group[['subcategory_id', 'skin_type_face', 'hair_issue', 'skin_type_body']].astype(str).apply(lambda x: ' '.join(x), axis=1)

    # Hitung TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform(attributes)

    # Tampilkan nilai bobot dari hasil perhitungan TF-IDF
    print("TF-IDF weights:")
    words = tfidf_vectorizer.get_feature_names_out()
    for i, doc in enumerate(tfidf_matrix.toarray()):
        print(f"Product {group.iloc[i]['id']} : {group.iloc[i]['name']}:")
        for j, word in enumerate(words):
            print(f"{word}: {doc[j]:.2f}")
        print()

    # Lakukan iterasi melalui setiap produk sebagai query
    for i, query_index in enumerate(range(len(group))):
        # Ambil query dan lakukan reshape
        query = tfidf_matrix[query_index]

        # Hitung similaritas kosinus antara query dan semua produk
        cosine_similarities = cosine_similarity(query, tfidf_matrix).flatten()

        # Urutkan indeks produk berdasarkan similaritas kosinus
        similar_indices = cosine_similarities.argsort()[::-1]

        # Tampilkan hasil similaritas kosinus untuk setiap produk
        print(f"Query Product: {group.iloc[query_index]['id']} : {group.iloc[query_index]['name']}")
        top_product_names = []
        for j in similar_indices[:10]:  # Ambil 10 hasil teratas
            if j != query_index:
                top_product_names.append(group.iloc[j]['name'])
                print(f"Similarity with Product {group.iloc[j]['id']} - {group.iloc[j]['name']}: {cosine_similarities[j]:.2f}")

        # Cari nilai bobot paling tertinggi-terendah
        max_weight_index = tfidf_matrix[query_index].toarray().argmax()
        min_weight_index = tfidf_matrix[query_index].toarray().argmin()
        max_weight_product = group.iloc[max_weight_index]['id']
        min_weight_product = group.iloc[min_weight_index]['id']
        print(f"Highest weight product: {max_weight_product} : {group.iloc[max_weight_index]['name']}, Weight: {tfidf_matrix[query_index].toarray().max():.2f}")
        print(f"Lowest weight product: {min_weight_product} : {group.iloc[min_weight_index]['name']}, Weight: {tfidf_matrix[query_index].toarray().min():.2f}")

        # Cari hasil tingkat kemiripan yang mendekati 1
        for j, similarity in enumerate(cosine_similarities):
            if similarity > 0.95 and j != query_index:  # Ubah threshold sesuai kebutuhan
                print(f"High similarity with Product {group.iloc[j]['id']} : {group.iloc[j]['name']}: {similarity:.2f}")
                break  # Keluar dari loop setelah menemukan tingkat kemiripan yang mendekati 1
        print()

        # Tampilkan daftar nama 10 produk teratas
        print(30*"=")
        print("Top 10 Recommended Products:")
        for name in top_product_names:
            print(name)
        print()

In [8]:
# Iterasi melalui setiap kelompok
for category_id, group in grouped_data:
    print(f"Category ID: {category_id}")
    # Hitung similaritas untuk setiap kelompok
    calculate_similarity(group)

Category ID: 1
TF-IDF weights:
Product 46 : Face Tonic Lemon:
cabang: 0.00
kering: 0.00
ketombe: 0.00
kombinasi: 0.00
minyak: 0.00
none: 0.00
normal: 1.00
rontok: 0.00
sensitif: 0.00

Product 46 : Face Tonic Lemon:
cabang: 0.00
kering: 0.00
ketombe: 0.00
kombinasi: 0.54
minyak: 0.84
none: 0.00
normal: 0.00
rontok: 0.00
sensitif: 0.00

Product 46 : Face Tonic Lemon:
cabang: 0.00
kering: 0.00
ketombe: 0.66
kombinasi: 0.00
minyak: 0.00
none: 0.00
normal: 0.75
rontok: 0.00
sensitif: 0.00

Product 46 : Face Tonic Lemon:
cabang: 0.00
kering: 0.00
ketombe: 0.00
kombinasi: 0.00
minyak: 0.00
none: 0.00
normal: 0.71
rontok: 0.00
sensitif: 0.70

Product 46 : Face Tonic Lemon:
cabang: 0.00
kering: 0.00
ketombe: 0.56
kombinasi: 0.55
minyak: 0.00
none: 0.00
normal: 0.00
rontok: 0.00
sensitif: 0.63

Product 47 : Face Tonic Bengkoang:
cabang: 0.00
kering: 0.00
ketombe: 0.00
kombinasi: 0.00
minyak: 0.56
none: 0.00
normal: 0.83
rontok: 0.00
sensitif: 0.00

Product 47 : Face Tonic Bengkoang:
cabang: 0.00